In [1]:
import glob
import os

# Numpy for numpy.arrays
import numpy as np
import pandas as pd
# Include ITK for DICOM reading.
import itk

# Include pydicom_seg for DICOM SEG objects
import pydicom
import pydicom_seg

# for downloading data from TCIA
from tcia_utils import nbia
import nibabel as nib
# This is the most common import command for itkWidgets.
#   The view() function opens an interactive viewer for 2D and 3D
#   data in a variety of formats.
from itkwidgets import view
import matplotlib as plt

# imports for monai
import torch
from monai.data import decollate_batch
from monai.bundle import ConfigParser, download
from monai.config import print_config

import SimpleITK as sitk

from tqdm import tqdm

2024-03-18 21:48:23,467:WARNING:From c:\Users\Joel Fischer\.conda\envs\segmentationenv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [2]:
#import the pirads file and change the rownames:
pirads = pd.read_excel('Pirads_updated_age_PSA.xlsx')
pirads.rename(columns={'Unnamed: 0': 'lesion_name'},inplace=True)
pirads.index = list(pirads['lesion_name'])
pirads.drop('lesion_name',axis=1,inplace=True)

In [3]:
translation = pd.read_excel('C:\\Users\\Joel Fischer\\Documents\\Masterarbeit\\Studie\\Projekt\\Auswertungsdaten\\Benchmark\\translation.xlsx')
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
translation.drop(8,inplace=True) #remove the bell_0027 case as its metadata is missing. 

In [4]:
#Get all lesion filenames:
lesion_filenames = []
directory = 'E:\\ksa_study_data'
for path, dirs, filenames in os.walk(directory):
    for f in filenames:
        if f.endswith('.nii.gz'):
            #get only the files of the 67 Cases we have biopsied totally:
            name = f.split('_')[0].split('-')[0] + '_' + f.split('_')[0].split('-')[1]
            #we need to manual add the bell_0045 Case as it is missing in the translation:
            if name in list(translation['Case']) or name == 'bell_0045':
                #we only want lesions of t2w images, not full prostate segmentations or t2w images:
                if f.split('.')[0].split('_')[-1] != 'none':
                    if f.split('_')[1] == 't2w':
                        if f.split('_')[2] != 'pro':
                            lesion_filenames.append(f)

In [150]:
def get_segmentation_array_and_metadata(filename):
    seg = nib.load(filename)
    return (seg.get_fdata(),seg.affine, seg.header)


def get_list_of_segmentations(filename_list):
    segmentation_list = []
    Case = filename_list[0].split('-')[0]
    Case_name = filename_list[0].split('_')[0]
    affine = get_segmentation_array_and_metadata(f"E:/ksa_study_data/{Case}/{Case_name}/{filename_list[0]}")[1]
    header = get_segmentation_array_and_metadata(f"E:/ksa_study_data/{Case}/{Case_name}/{filename_list[0]}")[2]

    for filename in range(len(filename_list)):
        Case = filename_list[filename].split('-')[0]
        Case_name = filename_list[filename].split('_')[0]
        seg = get_segmentation_array_and_metadata(f"E:/ksa_study_data/{Case}/{Case_name}/{filename_list[filename]}")[0]
        segmentation_list.append(seg)
    return (segmentation_list,affine,header)

def get_filenames_of_Case(Case_segmentations):
    n_pcai100_auto = []
    n_rad1_rad = []
    n_rad1_ass = []
    n_rad2_rad = []
    n_rad2_ass = []
    n_rad3_rad = []
    n_rad3_ass = []

    for segmentation in Case_segmentations:
        if segmentation.split('.')[-3].split('_')[-1] == 'auto' and segmentation.split('.')[-3].split('_')[-2] == 'pcai100':
            n_pcai100_auto.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'rad' and segmentation.split('.')[-3].split('_')[-2] == 'rad1':
            n_rad1_rad.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'ass' and segmentation.split('.')[-3].split('_')[-2] == 'rad1':
            n_rad1_ass.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'rad' and segmentation.split('.')[-3].split('_')[-2] == 'rad2':
            n_rad2_rad.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'ass' and segmentation.split('.')[-3].split('_')[-2] == 'rad2':
            n_rad2_ass.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'rad' and segmentation.split('.')[-3].split('_')[-2] == 'rad3':
            n_rad3_rad.append(segmentation)
        if segmentation.split('.')[-3].split('_')[-1] == 'ass' and segmentation.split('.')[-3].split('_')[-2] == 'rad3':
            n_rad3_ass.append(segmentation)
    return(n_pcai100_auto,n_rad1_rad,n_rad1_ass,n_rad2_rad,n_rad2_ass,n_rad3_rad,n_rad3_ass)

def make_binary_array(seg_array):
    for i in range(seg_array.shape[0]):
        for j in range(seg_array.shape[1]):
            for k in range(seg_array.shape[2]):
                #if a voxel only belongs to array1:
                if seg_array[i,j,k] > 1:
                    seg_array[i,j,k] = 1
    return seg_array

def change_pixel_value(seg_array,pixel_value):
    for i in range(seg_array.shape[0]):
        for j in range(seg_array.shape[1]):
            for k in range(seg_array.shape[2]):
                #if a voxel only belongs to array1:
                if seg_array[i,j,k] == 1:
                    seg_array[i,j,k] = pixel_value
    return seg_array


def summed_segmentations(segmentation_list):

    while len(segmentation_list) > 1: #only add them up, if there are at least 2 segmentations left:

        summed_seg = np.where(segmentation_list[0] != 1, 0, 1) + np.where(segmentation_list[1] != 0, 1, 0)
        summed_seg_binary = make_binary_array(summed_seg)
        del segmentation_list[1]
        segmentation_list[0] = summed_seg_binary
    return segmentation_list[0]



def summed_segmentations_nonbinary(segmentation_list,segdata):

    #create an empty dataframe with the same dimensions(try and except because tri20 has no fuseAI segmentation)

    final_segmentation =  segdata * 0
    seg_number = 1

    #if there is only one lesion:
    if len(segmentation_list) == 1:
        final_segmentation = segmentation_list[0] * 1
        return final_segmentation

    for seg_number in range(len(segmentation_list)): #only add them up, if there are at least 2 segmentations left:
        #loop trough all pixels:
        for i in range(segmentation_list[0].shape[0]):
            for j in range(segmentation_list[0].shape[1]):
                for k in range(segmentation_list[0].shape[2]):

                    if final_segmentation[i,j,k] != 0:
                        #when the first and second segmentations are both positive:
                        if segmentation_list[seg_number][i,j,k] != 0:
                            #for an overlap:
                            final_segmentation[i,j,k] = 8
                        #when the first is positive and the second negative we want the value of the first:
                        if segmentation_list[seg_number][i,j,k] == 0:
                            final_segmentation[i,j,k] = final_segmentation[i,j,k]

                    if final_segmentation[i,j,k] == 0:
                        #when both are negative
                        if segmentation_list[seg_number][i,j,k] == 0:
                            final_segmentation[i,j,k] = 0
                        #when the first is negative and the second positive we want the value of the second:
                        if segmentation_list[seg_number][i,j,k] != 0:
                            final_segmentation[i,j,k] = seg_number + 1

    return final_segmentation


def pool_pooled_segmenations(segmentation_list,segdata):
    #create an empty dataframe with the same dimensions(try and except because tri20 has no fuseAI segmentation)

    final__segmentation =  segdata * 0
    segmentation_number = 0
    for segmentation_number in range(len(segmentation_list)): #only add them up, if there are at least 2 segmentations left:
        #if there is a pooled segmentation:
        if not not len(segmentation_list[segmentation_number]):

            #loop trough all pixels:
            for i in range(segmentation_list[segmentation_number].shape[0]):
                for j in range(segmentation_list[segmentation_number].shape[1]):
                    for k in range(segmentation_list[segmentation_number].shape[2]):
                        
                        #when the second segmentation is positive:
                        if segmentation_list[segmentation_number][i,j,k] == segmentation_number + 1:
                            #when the pooling segmentation is not zero, we make it an 8 for overlap:
                            if final__segmentation[i,j,k] != 0:
                                final__segmentation[i,j,k] = 8
                            #when the pooling segmentation is zero, we add the value of the second segmentation:
                            if final__segmentation[i,j,k] == 0:
                                final__segmentation[i,j,k] = segmentation_number + 1
                        #when the second segmentation is negative:
                        if segmentation_list[segmentation_number][i,j,k] == 0:
                            continue

        segmentation_number = segmentation_number + 1

    return final__segmentation

def load_image_meatadata(Reader, lesion_name):
    import_folder = 'E:\\ksa_study_data\\' + lesion_name.split('_')[0]
    lesion = lesion_name.split('_')[2] + '-' + lesion_name.split('_')[3]
    Case = lesion_name.split('_')[0] + '-' + lesion_name.split('_')[1]
    try:
        pirads_value = str(pirads.loc[lesion_name.split('_')[0] + '_'+ lesion_name.split('_')[1] + '_' + lesion + '_' + Reader + '_t2w','PIRADS'])
    except:
        return None
    
    filename = import_folder + '\\'+ Case + '\\'+ Case + '_t2w_' + lesion + '--p-'+ str(pirads_value) +'--g-0_' + Reader + '.nii.gz'
    seg = nib.load(filename)
    return (seg.affine, seg.header)

def get_metadatafile(n_xlayers,n_zlayers):
    #for the bellinzona files
    if n_zlayers == 24 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/bell_0001_pcai100_auto_t2w_test.nii")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    
    if n_zlayers == 26 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/bell-0017_t2w_les-0--p-1--g-0_pcai100_auto_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    
    if n_zlayers == 24 and n_xlayers == 576:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/bell-0033_t2w_les-0--p-1--g-0_pcai100_auto_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    
    #for the ksa files
    if n_zlayers == 22 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/ksa3-0006_t2w_les-0--p-1--g-0_pcai100_auto_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    

    if n_zlayers == 25 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/ksa3-0751_t2w_les-0--p-1--g-0_pcai100_auto_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    
    if n_zlayers == 23 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/ksa3-0945_t2w_les-0--p-1--g-0_pcai100_auto_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)
    
    #for the triemli files
    if n_zlayers == 20 and n_xlayers == 512:
        seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/tri-0002_t2w_les-0--p-4--g-0_rad2_ass_test.nii.gz")
        segaffine = seg.affine
        segheader = seg.header
        segdata = seg.get_fdata()
        return (segdata, segaffine, segheader)


    

# For each Case, put all segmentations into a single file

 For each Case, each reader and annotation type gets a specific colour, which is consistend across Cases. Also the 
 overlaps have a specific colour.

 Colours: (so the pixel value is consistent)
 Fusion-AI: 1
 rad1_rad: 2
 rad1_ass: 3
 rad2_rad: 4
 rad2_ass: 5
 rad3_rad: 6
 rad3_ass: 7
 overlap: 8

In [158]:
#For each Case, Make an individual segmentation file:

#Get all the Cases:
Cases = list(translation['Case'])
Cases.append('bell_0045')

#load a test file, which we need to copy to have a working filemetadata to save the segmentations:
    # seg = nib.load("E:/ksa_study_data/converted_files/pooled_segmentations/bell_0001_pcai100_auto_t2w_test.nii")
    # segaffine = seg.affine
    # segheader = seg.header
    # segdata = seg.get_fdata()

for Case in tqdm(Cases):
    
    #Get the Case name with the '-':
    Case_name = Case.split('_')[0] + '-' + Case.split('_')[1]
    #Get only the filenames for the specific Case:
    Case_segmentations = [i for i in lesion_filenames if i.startswith(Case_name)]

    #If there is more than one predicted lesion for a reader, we pool the prediction. 
    filenames_of_segs_to_get_pooled = get_filenames_of_Case(Case_segmentations)
    
    test_list = []
    combined_Reader_segmentations_list = []

    #For each Entry of this dataframe, calculate the IoU:
    for i, ReaderX_filename in enumerate(filenames_of_segs_to_get_pooled):
        #Get the combined segmentation for this Reader (row):
        if not not ReaderX_filename: #when the list is not empty, so there is a lesion of this reader:
            Reader = ReaderX_filename[0].split('_')[-2].split('.')[0]
            annot_type = ReaderX_filename[0].split('_')[-1].split('.')[0]

            #For each Reader, get the pooled lesions and add it to a list with the pixel vlaues changed according to the values in the markdown.
            seg_info = get_list_of_segmentations(filenames_of_segs_to_get_pooled[i])
            #get the metadata:
            segdata, affine, header = get_metadatafile(seg_info[0][0].shape[0],seg_info[0][0].shape[2])

            #Make a nonbinary segmentation mask for each Reader:
            Case_pooled_Reader_segmentation_nonbinary = summed_segmentations_nonbinary(seg_info[0],segdata)
            test_list.append(Case_pooled_Reader_segmentation_nonbinary)
            #for each Reader, save the combined file of only this reader:
            seg_converted = nib.Nifti1Image(Case_pooled_Reader_segmentation_nonbinary, affine, header)
            save_path = f"E:/ksa_study_data/converted_files/pooled_segmentations/{Case}"
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            nib.save(seg_converted, f"{save_path}/{Case}_{Reader}_{annot_type}_t2w_.nii")


            #Make a binary segmentation mask for the file where all Readers are combined:
            auto_segmentations = summed_segmentations(seg_info[0])
            Case_pooled_segmentation_binary = make_binary_array(Case_pooled_Reader_segmentation_nonbinary)
            Case_pooled_segmentation_binary = change_pixel_value(Case_pooled_segmentation_binary,i+1)
            combined_Reader_segmentations_list.append(Case_pooled_segmentation_binary)

        else:
            combined_Reader_segmentations_list.append([])

    #pool the pooled lesions segmentations into a single file:
    Case_segmentation = pool_pooled_segmenations(combined_Reader_segmentations_list,segdata)
    
    #create pooled pooled lesions file:
    seg_converted_pooled = nib.Nifti1Image(Case_segmentation, affine, header)
    save_path = f"E:/ksa_study_data/converted_files/pooled_segmentations/{Case}"

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    nib.save(seg_converted_pooled, f"{save_path}/{Case}_t2w_.nii")
    

  0%|          | 0/67 [00:00<?, ?it/s]

0
1
0
1


  1%|▏         | 1/67 [01:17<1:25:40, 77.89s/it]

0
1


  3%|▎         | 2/67 [02:13<1:10:18, 64.90s/it]

...

 97%|█████████▋| 65/67 [9:28:06<03:08, 94.40s/it]

0
1
2
3
4
5
0
1
0
1
2
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3


 99%|█████████▊| 66/67 [9:32:54<02:32, 152.39s/it]

0
1
0
1
2
3
0
1
2
3
0
1
2
3


100%|██████████| 67/67 [9:35:46<00:00, 515.61s/it]
